<a href="https://colab.research.google.com/github/amshriva810/SkyImages/blob/main/Aarav_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/dataset_split/train/class1
!ls /content/drive/MyDrive/dataset_split/validation/class1


20200919.14-00-00.jpg  20200919.22-00-00.jpg  20200920.16-00-00.jpg  20200920.20-00-00.jpg
20200919.15-00-00.jpg  20200919.23-00-00.jpg  20200920.17-00-00.jpg  20200920.21-00-00.jpg
20200919.16-00-00.jpg  20200920.14-00-00.jpg  20200920.18-00-00.jpg  20200920.22-00-00.jpg
20200919.17-00-00.jpg  20200920.15-00-00.jpg  20200920.19-00-00.jpg  20200920.23-00-00.jpg
20200919.14-00-00.jpg  20200919.22-00-00.jpg  20200920.16-00-00.jpg  20200920.20-00-00.jpg
20200919.15-00-00.jpg  20200920.14-00-00.jpg  20200920.17-00-00.jpg  20200920.21-00-00.jpg
20200919.17-00-00.jpg  20200920.15-00-00.jpg  20200920.19-00-00.jpg  20200920.23-00-00.jpg


In [ ]:
import os
import shutil
import random

def split_data(source_dir, train_dir, val_dir, split_ratio=0.8):
    """
    Splits data from source_dir into train_dir and val_dir with the given split_ratio.

    Parameters:
    - source_dir: The source directory containing class subdirectories.
    - train_dir: The directory where the training data will be stored.
    - val_dir: The directory where the validation data will be stored.
    - split_ratio: The ratio of data to use for training (default is 0.8).
    """
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    if not os.path.exists(val_dir):
        os.makedirs(val_dir)

    for class_name in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_name)
        if os.path.isdir(class_path):
            train_class_path = os.path.join(train_dir, class_name)
            val_class_path = os.path.join(val_dir, class_name)
            os.makedirs(train_class_path, exist_ok=True)
            os.makedirs(val_class_path, exist_ok=True)

            files = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
            random.shuffle(files)
            split_index = int(len(files) * split_ratio)

            train_files = files[:split_index]
            val_files = files[split_index:]

            for file in train_files:
                shutil.copy2(os.path.join(class_path, file), os.path.join(train_class_path, file))
            for file in val_files:
                shutil.copy2(os.path.join(class_path, file), os.path.join(val_class_path, file))

# Define the paths
source_dir = '/content/drive/MyDrive/dataset/train'
train_dir = '/content/drive/MyDrive/dataset_split/train'
val_dir = '/content/drive/MyDrive/dataset_split/validation'

# Split the data
split_data(source_dir, train_dir, val_dir, split_ratio=0.8)




In [ ]:
# Verify the directories in train and validation
!ls /content/drive/MyDrive/dataset_split/train
!ls /content/drive/MyDrive/dataset_split/validation

# List a few files from one class directory in train and validation
!ls /content/drive/MyDrive/dataset_split/train/class1 | head -n 10
!ls /content/drive/MyDrive/dataset_split/validation/class1 | head -n 10


class1	class2	class3	class4
class1	class2	class3	class4
20200919.14-00-00.jpg
20200919.15-00-00.jpg
20200919.16-00-00.jpg
20200919.17-00-00.jpg
20200919.22-00-00.jpg
20200919.23-00-00.jpg
20200920.14-00-00.jpg
20200920.15-00-00.jpg
20200920.16-00-00.jpg
20200920.17-00-00.jpg
20200919.14-00-00.jpg
20200919.15-00-00.jpg
20200919.16-00-00.jpg
20200919.17-00-00.jpg
20200919.22-00-00.jpg
20200919.23-00-00.jpg
20200920.14-00-00.jpg
20200920.15-00-00.jpg
20200920.16-00-00.jpg
20200920.17-00-00.jpg


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset_split/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset_split/validation',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 104 images belonging to 4 classes.
Found 78 images belonging to 4 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:



train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset_split/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset_split/validation',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 104 images belonging to 4 classes.
Found 78 images belonging to 4 classes.


In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False)
# Load the pre-trained ResNet50 model without the top (fully connected) layers
#base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
#x = Dropout(0.5)(x)  # Dropout to prevent overfitting
predictions = Dense(4, activation='softmax')(x)  # Assuming 4 classes

# Combine base model and custom layers into a new model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


<ipython-input-9-0cfa4d29dbf6>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:




history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 36s 8s/step - accuracy: 0.1667 - loss: 1.7122 - val_accuracy: 0.5128 - val_loss: 1.1074
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.4839 - loss: 1.1275 - val_accuracy: 0.5000 - val_loss: 0.9731
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.4732 - loss: 1.0091 - val_accuracy: 0.7564 - val_loss: 0.7530
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.7458 - loss: 0.7171 - val_accuracy: 0.5641 - val_loss: 0.7472
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.7135 - loss: 0.7424 - val_accuracy: 0.8718 - val_loss: 0.5856
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.8032 - loss: 0.6187 - val_accuracy: 0.8590 - val_loss: 0.4609
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.8143 - loss: 0.5446 - val_accuracy: 0.8590 - val_loss: 0.4239
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step - accuracy: 0.7868 - loss: 0.4883 - val_accuracy: 0.8718 - val_loss: 0.4227
Epoch 9/20
4/4 ━━━━

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(val_generator)
print(f'Validation Accuracy: {accuracy*100:.2f}%')

# Save the model
model.save('sky_classification_model.h5')


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 976ms/step - accuracy: 0.9897 - loss: 0.2106


Validation Accuracy: 98.72%


In [ ]:
# Save the model in the recommended format
# Define the path in Google Drive where you want to save the model
model_save_path = '/content/drive/MyDrive/sky_classification_model_100percentval_acc.keras'

# Save the Keras model in TensorFlow’s native format
model.save(model_save_path, save_format='keras')

print(f"Model saved to: {model_save_path}")

!ls /content/drive/MyDrive/




Model saved to: /content/drive/MyDrive/sky_classification_model_100percentval_acc.keras
 20dec06.apinene.dilution.size.dist.gsheet
 A52_H_07_Zelenyuk.pptx
 Aarav_MidColumbia_ScienceFair_Wildfires.gslides
'Aarav_presentation 1.gslides'
'AMZ SVTAG shareable 190306 IOP2 tracers for Manish.gsheet'
 ARM-AMF2-Proposal
'Colab Notebooks'
'Copy of MS_response_SOA.gdoc'
'Copy of NCAT-CRC Proposal Draft #2.docx'
 dataset
 dataset_split
'FACL_Shrivastava_Wu_23367 revised.gdoc'
 Figures_v1.gdoc
 GRC_Les_Diablerets_Manish.zip
 labels.txt.gdoc
'manish est.pdf'
 ManishShrivastava_Seminar_2024.pptx
 MANISH_SHRIVESTAVA_REPAIRS.PDF
 Manuscript_v1.gdoc
 model.tflite
 MS_CAM_reviews
 mshrivastava_pic.jpeg
'New folder.zip'
'poster_RE3_Shrivastava_final (1).gslides'
 poster_RE3_Shrivastava_final.gslides
 Preetipic.jpg
'Project proposal.gdoc'
'Responses of Shrivastava et al. Rev. Geophys..gdoc'
 Ropes.zip
 Shrivastava_ASR_Plenary_2023.pptx
'Shrivastava PV Proposal (25).pdf'
'Shrivastava PV Proposal (27).pdf'


In [ ]:
# Load the saved model from Google Drive
model = tf.keras.models.load_model('/content/drive/MyDrive/sky_classification_model_100percentval_acc.keras')  # or .keras


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
from PIL import Image
import numpy as np

# Upload the image to Google Colab
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload an image file

# Load and preprocess the image
image = Image.open(list(uploaded.keys())[0])  # Open the uploaded image
image = image.resize((224, 224))  # Resize image to match model input size

# Convert the image to a NumPy array and normalize (scale pixel values to [0, 1])
image_array = np.array(image) / 255.0

# Add a batch dimension (because the model expects input with shape [batch_size, height, width, channels])
image_array = np.expand_dims(image_array, axis=0)

print(f"Image shape after preprocessing: {image_array.shape}")


Saving 20180819.18.jpg to 20180819.18.jpg
Image shape after preprocessing: (1, 224, 224, 3)


In [ ]:
# Make predictions on the new image
predictions = model.predict(image_array)

# The model returns an array of probabilities; find the class with the highest probability
predicted_class_index = np.argmax(predictions)

print(f"Predicted Class Index: {predicted_class_index}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Class Index: 2


In [ ]:
from google.colab import files

# Upload labels.txt from your local machine
uploaded = files.upload()

# This will prompt you to upload the file from your local machine.
# Once uploaded, you can access the file as follows:




Saving labels.txt to labels.txt


In [ ]:
# Load the labels.txt file
with open('labels.txt', 'r') as f:
    labels = f.read().splitlines()

# Check the loaded labels
print(labels)


['1', '2', '3', '4']


In [ ]:
# Assuming you have a preprocessed image and model ready
predictions = model.predict(image_array)

# Get the index of the class with the highest probability
predicted_class_index = np.argmax(predictions)

# Map the predicted index to the class label
predicted_label = labels[predicted_class_index]
print(f"Predicted Label: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Predicted Label: 3


In [ ]:
!cp /content/model.tflite /content/drive/MyDrive/

cp: cannot stat '/content/model.tflite': No such file or directory
